In [15]:
#!pip3 install selenium

import requests, time, random
from bs4 import BeautifulSoup
from selenium import webdriver

import pandas as pd
from typing import List
from parsel import Selector

#!pip3 install simplelist
#from simplelist import listfromtxt
#from simplelist import txtfromlist

def write(file,data):
    f = open(file, "a")
    for d in data:
        f.write(d+"\n")
    f.close()
def read(file):
    f = open(file, "r")
    data=f.read().split("\n")
    f.close()
    return data

def browser():
    options = webdriver.ChromeOptions()
    options.add_experimental_option("useAutomationExtension", False)
    options.add_experimental_option("excludeSwitches",["enable-automation"])
    
    driver_path = './chromedriver'
    driver = webdriver.Chrome(executable_path=driver_path,options=options)
    driver.get("https://www.linkedin.com/login/")
    return driver


def login():
    file = open("./config.txt")
    line = file.readlines()
    username = line[0]
    password = line[1]
    elementID = driver.find_element_by_id('username')
    elementID.send_keys(username)
    elementID = driver.find_element_by_id('password')
    elementID.send_keys(password)
    elementID.submit()
    return True

def profile_urls(ids_file="ids1.txt"):
    driver.get('https://www.linkedin.com/school/fastnu/people/?educationEndYear=2019&facetGeoRegion=au%3A0')
    rep = 2 #determine the rep enough to scroll all the page
    last_height = driver.execute_script("return document.body.scrollHeight")
    for i in range(rep):
      driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
      time.sleep(5)
      new_height = driver.execute_script("return document.body.scrollHeight")
      if new_height == last_height:
        break
      new_height = last_height

    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')
    pav = soup.find('div', {'class' : 'artdeco-card pv5 pl5 pr1 mt4'})
    all_links = pav.find_all('a', {'class' : 'ember-view link-without-visited-state'})    
    profilesID = []
    for link in all_links:
      profilesID.append(link.get('href'))
    
    print(len(profilesID))
    write(ids_file,profilesID)
    
    return profilesID

def get_profiles(Lists,driver):

    names=[]
    Current_Company =[]
    Current_Job=[]
    Total_year=[]
    Total_Experience=[]
    job_location=[]
    
    
    Last_University=[]
    Last_degree=[]
    Graduation_start_year=[]
    Graduation_end_year=[]
    
    
    
    skills_set=[]
    ProfileIDS=[]
    skl1=[]
    skl2=[]
    skl3=[]
    
    
    
    
    import random
      
    
    batch_size=10
    counter=0
    
    for profileID in Lists:
      counter+=1
      if(counter==batch_size):
          counter=0
          driver.quit()
          driver=browser()
          login()
      fulllink = 'https://www.linkedin.com' + profileID
      driver.get(fulllink)
      time.sleep(random.randint(5, 8))
      src = driver.page_source
      soup = BeautifulSoup(src, 'lxml')
      
      total_height = int(driver.execute_script("return document.body.scrollHeight"))
    
      for i in range(1, total_height, 5):
            driver.execute_script("window.scrollTo(0, {});".format(i))
    
        # assigning the source code for the webpage to variable sel
      sel = Selector(text=driver.page_source)
      
      #"""
      
      def get_education(sel,soup):
          print("Education")
          universities=[]
          degrees=[]
          st_years=[]
          end_years=[]
          educations=sel.xpath('//*[@id="education-section"]/ul/li/text()').extract()
          for education in educations:
              print(education)
              universities.append(education.xpath('//*[@class="pv-entity__degree-info"]/h3/text()').extract())
              print(education.xpath('//*[@class="pv-entity__degree-info"]/h3/text()').extract())
              degrees.append(education.xpath('//*[@class="pv-entity__degree-name"]/span[@class="pv-entity__comma-item"]/text()').extract())
              st_years.append(education.xpath('//*[@class="pv-entity__dates"]/span[2]/time[1]/text()').extract())
              end_years.append(education.xpath('//*[@class="pv-entity__dates"]/span[2]/time[2]/text()').extract())
          
          print(universities,degrees,st_years,end_years)
          return universities,degrees,st_years,end_years
      universities,degrees,st_years,end_years=get_education(sel,soup)
      #"""    
      
    
      name = sel.xpath('//*[starts-with(@class, "text-heading-xlarge inline t-24 v-align-middle break-words")]/text()').extract_first()
      if name:
          name = name.strip()

      names.append(name)
          #print(name)
        
      #scraping companies
      company = sel.xpath('//*[starts-with(@class, "pv-entity__secondary-title t-14 t-black t-normal")]/text()').extract_first()
      if company:
        company = company.strip()
      Current_Company.append(company)
    
    
      job_title = sel.xpath('//*[starts-with(@class, "t-16 t-black t-bold")]/text()').extract_first()
      if job_title:
          job_title = job_title.strip()
      Current_Job.append(job_title)
          #print(jobtitle)
        
      # job location
      job_locations = sel.xpath('//*[starts-with(@class, "pv-entity__location t-14 t-black--light t-normal block")]/span[2]/text()').extract_first()
      #job_locations = job.find("span", class_="job-result-card__location").text
      if job_locations:
          job_locations = job_locations.strip()
      job_location.append(job_locations)
      # print(Job location)
      #else: 
         #job_locations = sel.xpath('//*[starts-with(@class, "text-body-small inline t-black--light break-words")]/text()').extract_first()
             #if job_locations:
                 #location = job_locations.strip()
             #job_location.append(job_locations)
          # print(location)
          
      #//*[@id="education-section"]/ul/li/div/div/a/div[2]/div/h3
      
      #education=sel.xpath('//*[@id="education-section"]')
      educations=sel.xpath('//*[@id="education-section"]/ul/li/div/div/a/div[2]/div/h3/text()')
      for u in educations:
        print(u.extract())
          
            
            
      University = sel.xpath('//*[starts-with(@class, "pv-entity__school-name t-16 t-black t-bold")]/text()').extract_first()
      if University:
          University = University.strip()
      Last_University.append(University)
          # print(location)
        
        
              
      degree = sel.xpath('//*[starts-with(@class, "pv-entity__comma-item")]/text()').extract_first()
      if degree:
          degree = degree.strip()
      Last_degree.append(degree)
          # print(last Degree)
        
      Grad_start_year = sel.xpath('//*[starts-with(@class, "pv-entity__dates t-14 t-black--light t-normal")]/span[2]/time[1]/text()').extract_first()
      if Grad_start_year:
         Grad_start_year = Grad_start_year.strip()
      Graduation_start_year.append(Grad_start_year)
          # print(Grad_end_year)
        
        
      Grad_end_year = sel.xpath('//*[starts-with(@class, "pv-entity__dates t-14 t-black--light t-normal")]/span[2]/time[2]/text()').extract_first()
      if Grad_end_year:
         Grad_end_year = Grad_end_year.strip()
      Graduation_end_year.append(Grad_end_year)
          # print(Grad_end_year)
        
      
    
      skill_ary=[]
       # skills = sel.xpath('//*[@id="ember534"]/span').extract_first();
      for skill in sel.xpath('//*[starts-with(@class, "pv-skill-category-entity__name-text t-16 t-black t-bold")]'):
            #print(li.xpath('.//@href').get())
            print(skill.xpath('.//text()').get());
            skill_ary.append(skill.xpath('.//text()').get())
      if len(skill_ary) > 0:
        skl1.append(skill_ary[0].strip())
      if len(skill_ary) > 1:
        skl2.append(skill_ary[1].strip())
      if len(skill_ary) > 2:
        skl3.append(skill_ary[2].strip())
      #skills = sel.xpath('//*[starts-with(@class, "pv-skill-category-entity__name-text t-16 t-black t-bold")]//text()').extract_first()
     
     ##Graduation_year = sel.xpath('//*[starts-with(@class, "pv-entity__secondary-title "pv-entity__degree-name t-14 t-black t-normal")]/span()/span()/text()').extract_first()
     
      time.sleep(random.randint(2, 8))
      #time.sleep(5)  
      #for skill in skills:
       #     skills = sel.xpath('//*[starts-with(@class,"pv-skill-category-entity__name-text t-16 t-black t-bold")]/text()').extract()
      #skills_set.append(skills)
         
    
      Dates_Employed = sel.xpath('//*[starts-with(@class, "pv-entity__date-range t-14 t-black--light t-normal")]/span[2]/text()').extract_first()
      if Dates_Employed:
         Dates_Employed = Dates_Employed.strip()
      Total_year.append(Dates_Employed)
          # print(Dates_Employed)
        
        
      Experience = sel.xpath('//*[starts-with(@class, "pv-entity__bullet-item-v2")]/text()').extract_first()
      if Experience:
            Experience = Experience.strip()
      Total_Experience.append(Experience)
          # print(Experience)
         
            
            
      profileID = driver.current_url
      ProfileIDS.append(profileID)
         
    
      #Print what information is being write to the CSV file
      rows = [name,company,job_title,Total_year,Total_Experience,job_location,degree,Graduation_start_year,Graduation_end_year,skl1,skl2,skl3,University,profileID]
      
      
      if(counter==batch_size):
          print('Writing to CSV:', rows)
          df = pd.DataFrame(list(zip(names,Current_Company,Current_Job,Total_year,Total_Experience,job_location,Last_degree,Graduation_start_year,Graduation_end_year,skl1,skl2,skl3,Last_University,ProfileIDS)), 
                   columns =['Name', 'Current_Company', 'Current_Job', 'Total_year',   'Total_Experience',  'job_location',  'Last_degree' ,  'Graduation_start_year', 'Graduation_end_year','skill_1','skill_2','skill_3',  'Last_University', 'ProfileIDS'])
        
          #print(df)
          df.to_csv('./Scrapdata1111.csv', index=False)

    df = pd.DataFrame(list(zip(names,Current_Company,Current_Job,Total_year,Total_Experience,job_location,Last_degree,Graduation_start_year,Graduation_end_year,skl1,skl2,skl3,Last_University,ProfileIDS)),
                  columns =['Name', 'Current_Company', 'Current_Job', 'Total_year',   'Total_Experience',  'job_location',  'Last_degree' ,  'Graduation_start_year', 'Graduation_end_year','skill_1','skill_2','skill_3',  'Last_University', 'ProfileIDS'])
    df.to_csv('./Scrapdata1111.csv', index=False)





ids_file="ProfilesID13.txt"
driver=browser()
login()
#profilesID=profile_urls(ids_file)

Lists=read(ids_file)
get_profiles(Lists,driver)

Education
[] [] [] []
Swinburne University of Technology
National University of Computer and Emerging Sciences
Adamjee Government Science College

            Java
          

          Machine Learning
        

          Databases
        
Education
[] [] [] []
Macquarie University
National University of Computer and Emerging Sciences
Education
[] [] [] []
Deakin University
National University of Computer and Emerging Sciences

            Android Development
          

            ASP.NET MVC
          

            Software Testing
          
Education
[] [] [] []
Education
[] [] [] []
Education
[] [] [] []


WebDriverException: Message: chrome not reachable
  (Session info: chrome=94.0.4606.61)
